In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
warnings.filterwarnings('ignore')

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

# **Data Posh Cash**
---
* Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
* This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.
---
**Identifikasi Utama:**

* SK_ID_PREV: ID aplikasi kredit sebelumnya. Setiap baris mewakili snapshot saldo tertentu untuk pinjaman sebelumnya.
* SK_ID_CURR: ID aplikasi kredit saat ini. Ini memungkinkan Anda untuk menghubungkan data saldo POS/tunai dengan pinjaman atau aplikasi pelanggan saat ini.

**Informasi Saldo dan Pembayaran:**

* MONTHS_BALANCE: Bulan relatif dari snapshot saldo. Nilai negatif kemungkinan menunjukkan bulan-bulan sebelum titik referensi (0). Pada dasarnya, berapa bulan yang lalu saldo ini dicatat.
* CNT_INSTALMENT: Jumlah total angsuran untuk pinjaman.
* CNT_INSTALMENT_FUTURE: Jumlah angsuran yang tersisa untuk pinjaman pada saat snapshot.

**Status Pinjaman:**\
\
NAME_CONTRACT_STATUS: Status pinjaman pada saat snapshot. Nilai-nilai umum mungkin termasuk:
* Aktif: Pinjaman saat ini aktif.
* Selesai: Pinjaman telah sepenuhnya dilunasi.
* Ditandatangani: Kontrak ditandatangani tetapi pencairan mungkin belum terjadi.
* Disetujui: Aplikasi pinjaman disetujui.
* Ditolak: Aplikasi pinjaman ditolak.
* Status lain yang menunjukkan keterlambatan, dll.

**Informasi Keterlambatan:**

* SK_DPD: Jumlah hari keterlambatan (逾期天数) pada angsuran pada saat snapshot. Ini merupakan ukuran keterlambatan yang penting.
* SK_DPD_DEF: Jumlah hari keterlambatan (逾期天数) untuk periode terpanjang selama riwayat pinjaman hingga tanggal snapshot. Ini menangkap keterlambatan terburuk yang dialami pelanggan pada pinjaman ini.

**Informasi yang Diberikan Data Ini:**

* Perilaku Pembayaran Pelanggan: Anda dapat menganalisis bagaimana pelanggan membayar pinjaman POS/tunai mereka dari waktu ke waktu:
* Apakah mereka melakukan pembayaran tepat waktu (SK_DPD rendah)?
* Apakah mereka secara konsisten membayar angsuran (CNT_INSTALMENT, CNT_INSTALMENT_FUTURE) seperti yang disepakati?
* Bagaimana status pinjaman mereka berubah dari waktu ke waktu (NAME_CONTRACT_STATUS)?
* Pola Keterlambatan: Anda dapat mengidentifikasi pelanggan yang berisiko gagal bayar dengan melihat SK_DPD dan SK_DPD_DEF. Peningkatan nilai-nilai ini dapat mengindikasikan kesulitan keuangan.
* Kinerja Pinjaman: Anda dapat menilai kinerja keseluruhan pinjaman POS/tunai dengan menganalisis distribusi status dan keterlambatan.
* Segmentasi Pelanggan: Anda dapat segmen pelanggan berdasarkan perilaku pembayaran dan pola keterlambatan mereka untuk strategi pemasaran atau manajemen risiko yang ditargetkan.

**Pengamatan dari Cuplikan Data:**

* Sebagian besar pinjaman dalam cuplikan berstatus "Aktif."
* Beberapa pinjaman memiliki SK_DPD lebih besar dari 0, menunjukkan bahwa pelanggan telah melewatkan pembayaran.
* Beberapa baris terakhir menunjukkan contoh di mana CNT_INSTALMENT_FUTURE adalah 0, menunjukkan pinjaman mungkin mendekati penyelesaian atau telah lunas.
* Status "Selesai" di baris terakhir menunjukkan pinjaman yang lunas.

**Analisis Potensial:**

* Hitung rata-rata SK_DPD dan SK_DPD_DEF untuk status pinjaman yang berbeda.
* Lacak bagaimana SK_DPD berubah dari waktu ke waktu untuk pinjaman individu.
* Analisis hubungan antara CNT_INSTALMENT, CNT_INSTALMENT_FUTURE, dan MONTHS_BALANCE.
* Bangun model untuk memprediksi kemungkinan keterlambatan di masa depan berdasarkan fitur yang tersedia.

In [3]:
# df_poshcash = pd.read_csv('/content/drive/MyDrive/homecredit/POS_CASH_balance.csv')
df_poshcash = pd.read_csv('homecredit/POS_CASH_balance.csv')
df_poshcash

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
...,...,...,...,...,...,...,...,...
10001353,2448283,226558,-20,6.0,0.0,Active,843,0
10001354,1717234,141565,-19,12.0,0.0,Active,602,0
10001355,1283126,315695,-21,10.0,0.0,Active,609,0
10001356,1082516,450255,-22,12.0,0.0,Active,614,0


In [4]:
print(df_poshcash['SK_ID_PREV'].unique().shape)
print(df_poshcash['SK_ID_CURR'].unique().shape)

(936325,)
(337252,)


In [5]:
df_posh_cash = df_poshcash.drop('SK_ID_PREV', axis=1)

In [6]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df,
                        columns= categorical_columns,
                        dummy_na= nan_as_category       # Treat NaNs as category
                       )
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

nan_as_category = True

In [7]:
# 4.0 OneHotEncode (OHE) 'object' types in bureau

df_posh_cash, cat_cols = one_hot_encoder(df_posh_cash, nan_as_category= True)

In [8]:
num_aggregations = {
                     'MONTHS_BALANCE':          ['count', 'mean'],
                     'CNT_INSTALMENT':          ['mean'],
                     'CNT_INSTALMENT_FUTURE':   ['mean'],
                     'SK_DPD':                  ['mean'],
                     'SK_DPD_DEF':              ['mean']
                    }

In [9]:
cat_aggregations = {}
for cat in cat_cols:
    cat_aggregations[cat] = ['sum']

# 5.2.1    
cat_aggregations  

{'NAME_CONTRACT_STATUS_Active': ['sum'],
 'NAME_CONTRACT_STATUS_Amortized debt': ['sum'],
 'NAME_CONTRACT_STATUS_Approved': ['sum'],
 'NAME_CONTRACT_STATUS_Canceled': ['sum'],
 'NAME_CONTRACT_STATUS_Completed': ['sum'],
 'NAME_CONTRACT_STATUS_Demand': ['sum'],
 'NAME_CONTRACT_STATUS_Returned to the store': ['sum'],
 'NAME_CONTRACT_STATUS_Signed': ['sum'],
 'NAME_CONTRACT_STATUS_XNA': ['sum'],
 'NAME_CONTRACT_STATUS_nan': ['sum']}

In [11]:
# 5.3 Perform aggregation now on SK_ID_CURR:

grouped = df_posh_cash.groupby('SK_ID_CURR')
cash_agg = grouped.agg({**num_aggregations, **cat_aggregations})

In [12]:
cash_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in cash_agg.columns.tolist()])
cash_agg = cash_agg.reset_index()
cash_agg

,SK_ID_CURR,PREV_MONTHS_BALANCE_COUNT,PREV_MONTHS_BALANCE_MEAN,PREV_CNT_INSTALMENT_MEAN,PREV_CNT_INSTALMENT_FUTURE_MEAN,PREV_SK_DPD_MEAN,PREV_SK_DPD_DEF_MEAN,PREV_NAME_CONTRACT_STATUS_Active_SUM,PREV_NAME_CONTRACT_STATUS_Amortized debt_SUM,PREV_NAME_CONTRACT_STATUS_Approved_SUM,PREV_NAME_CONTRACT_STATUS_Canceled_SUM,PREV_NAME_CONTRACT_STATUS_Completed_SUM,PREV_NAME_CONTRACT_STATUS_Demand_SUM,PREV_NAME_CONTRACT_STATUS_Returned to the store_SUM,PREV_NAME_CONTRACT_STATUS_Signed_SUM,PREV_NAME_CONTRACT_STATUS_XNA_SUM,PREV_NAME_CONTRACT_STATUS_nan_SUM
0,100001,9,-72.555556,4.000000,1.444444,0.777778,0.777778,7,0,0,0,2,0,0,0,0,0
1,100002,19,-10.000000,24.000000,15.000000,0.000000,0.000000,19,0,0,0,0,0,0,0,0,0
2,100003,28,-43.785714,10.107143,5.785714,0.000000,0.000000,26,0,0,0,2,0,0,0,0,0
3,100004,4,-25.500000,3.750000,2.250000,0.000000,0.000000,3,0,0,0,1,0,0,0,0,0
4,100005,11,-20.000000,11.700000,7.200000,0.000000,0.000000,9,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337247,456251,9,-5.000000,7.875000,4.375000,0.000000,0.000000,7,0,0,0,1,0,0,1,0,0
337248,456252,7,-79.000000,6.000000,3.000000,0.000000,0.000000,6,0,0,0,1,0,0,0,0,0
337249,456253,17,-79.235294,6.705882,2.000000,0.294118,0.294118,15,0,0,0,2,0,0,0,0,0
337250,456254,20,-5.550000,14.900000,10.350000,0.000000,0.000000,20,0,0,0,0,0,0,0,0,0


In [13]:
cash_agg.to_csv('stat_posh_cash.csv', index=False)